In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, LayerNormalization, Dropout, Dense, GlobalAveragePooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import nltk
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, LayerNormalization, MultiHeadAttention, Input, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import os
from tensorflow.keras import regularizers
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics import f1_score


In [ ]:
# Log parameters
params = {
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "epochs": 10,
    "batch_size": 128
}

# Define the paths to your CSV files
csv_file_path_1 = './data/data.csv'
csv_file_path_2 = './data/AEcodiert240430_UTF8.csv'
csv_file_path_3 = './data/meddra_zkls.csv'
csv_file_path_4 = './data/meddra_zkls2.csv'

# Read the CSV files into DataFrames
try:
    df1 = pd.read_csv(csv_file_path_1, delimiter=';', encoding='utf-8')
    df2 = pd.read_csv(csv_file_path_2, delimiter=';', encoding='utf-8')
    df3 = pd.read_csv(csv_file_path_3, delimiter=';', encoding='utf-8')
    df4 = pd.read_csv(csv_file_path_4, delimiter=';', encoding='utf-8')

except FileNotFoundError as e:
    print(e)
    exit()

# Concatenate all the DataFrames
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Function to split and expand DataFrame
def split_and_expand_df(df):
    def split_row(row):
        llt_code_list = str(row['llt_code']).split(',')
        return pd.DataFrame({'llt_code': llt_code_list, 'ae_description': row['ae_description']})

    df = df.apply(split_row, axis=1)
    df = pd.concat(df.tolist(), ignore_index=True)
    return df

expanded_df = split_and_expand_df(df)

stop_words = set(stopwords.words('german'))
stemmer = SnowballStemmer('german')

def preprocess_german_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text, language='german')
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return words

expanded_df['ae_description'] = expanded_df['ae_description'].apply(preprocess_german_text)

# Tokenization and Sequencing
max_words = 10000
max_len = 150

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(expanded_df['ae_description'])

sequences = tokenizer.texts_to_sequences(expanded_df['ae_description'])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Prepare embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=expanded_df['ae_description'], vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")

for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# One-hot encode multiclass feature
one_hot_multiclass = MultiLabelBinarizer()
multi_labels = one_hot_multiclass.fit_transform(expanded_df['llt_code'])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, multi_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Check the shapes of the data
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of y_val: {y_val.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")



In [ ]:
# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256
num_heads = 4
ff_dim = 512
num_classes = len(one_hot_multiclass.classes_)  # Assuming y_train is one-hot encoded

# Model architecture with multi-head attention and causal masking
inputs = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)(inputs)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding_layer, embedding_layer)
attention_output = LayerNormalization(epsilon=1e-5)(attention_output)  
attention_output = Dropout(0.1)(attention_output)

# Adding the second LSTM layer
lstm_output = LSTM(ff_dim, return_sequences=True, recurrent_dropout=0.1)(attention_output)
lstm_output = LayerNormalization(epsilon=1e-7)(lstm_output)
lstm_output = Dropout(0.1)(lstm_output)

# Adding the second LSTM layer
lstm_output = LSTM(ff_dim, return_sequences=True, recurrent_dropout=0.1)(attention_output)
lstm_output = LayerNormalization(epsilon=1e-7)(lstm_output)
lstm_output = Dropout(0.1)(lstm_output)

#Adding CNN Layer
# First Conv1D layer with 512 filters and kernel size of 3
conv1 = Conv1D(filters=512, kernel_size=3, activation='relu')(lstm_output)
conv1 = MaxPooling1D(pool_size=2)(conv1)
# Second Conv1D layer with 256 filters and kernel size of 3
conv2 = Conv1D(filters=256, kernel_size=3, activation='relu')(conv1)
conv2 = MaxPooling1D(pool_size=2)(conv2)
# Adding the first hidden layer with Tanh activation
ff_output1 = Dense(ff_dim, activation='tanh')(lstm_output)
ff_output1 = LayerNormalization(epsilon=1e-7)(ff_output1)  # Slightly smaller epsilon
ff_output1 = Dropout(0.1)(ff_output1)

# Adding the second hidden layer with ReLU activation
ff_output2 = Dense(ff_dim, activation='relu')(ff_output1)
ff_output2 = LayerNormalization(epsilon=1e-7)(ff_output1)  # Slightly smaller epsilon
ff_output2 = Dropout(0.1)(ff_output2)

flat_output = GlobalAveragePooling1D()(ff_output2)
outputs = Dense(num_classes, activation='softmax')(flat_output)

# Compile the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=256)

# Evaluate the model
y_pred = np.argmax(model.predict(X_val), axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_pred)

# Classification report
print(classification_report(y_val, y_pred))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model.save('gemischtesModel.h5')


In [ ]:

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Log additional custom metrics
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

f1 = f1_score(y_test, y_pred_classes, average='weighted')
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
print('F1 Score: ', f1)
print('Precision: ', precision)
print('Recall: ', recall)

# Confusion matrix
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_classes.argmax(axis=1))
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')

# Save the model
model.save('lstm_multilabel_model_with_attention.h5')

# Print model summary
print(model.summary())

In [ ]:
#Predict llt_code
input_text = "Erkältung mit Husten und Halsschmerzen"

def preprocess_german_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = word_tokenize(text, language='german')  # Tokenize
    words = [stemmer.stem(word) for word in words if word not in stop_words]  # Remove stopwords and stem
    return words

# Assuming `input_text` is the new text you want to predict
input_text_processed = preprocess_german_text(input_text)
input_sequence = tokenizer.texts_to_sequences([input_text_processed])  # Use tokenizer from training
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len)  # Pad sequences

from tensorflow.keras.models import load_model
model = load_model('lstm_multilabel_model_with_attention.h5')

# Make prediction
predictions = model.predict(padded_input_sequence)

# Get predicted class index (for multiclass classification)
predicted_class_index = np.argmax(predictions)

# Map back to category label if necessary (using your LabelEncoder)
predicted_category = one_hot_multiclass.inverse_transform([predicted_class_index])

print(f"Predicted category: {predicted_category[0]}")

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_architecture.png', show_shapes=True)

In [ ]:

y_test_predictions = model.predict(X_test)
pred_test=np.argmax(y_test_predictions, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Classification Metrics
accuracy = accuracy_score(y_test_labels, pred_test)
precision = precision_score(y_test_labels, pred_test, average='weighted')
recall = recall_score(y_test_labels, pred_test, average='weighted')
f1score = f1_score(y_test_labels, pred_test, average='weighted')

print(f"Accuracy = {round(accuracy, 4)}")
print(f"Precision = {round(precision, 4)}")
print(f"Recall = {round(recall, 4)}")
print(f"F1 Score = {round(f1score, 4)}")

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


# Convert y_test to class labels if it's in one-hot encoded format
y_test_labels = np.argmax(y_test, axis=1)

# Convert model predictions to class labels
y_test_predictions_labels = np.argmax(y_test_predictions, axis=1)

# Now calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_test_predictions_labels)
print(conf_matrix)

from tensorflow.keras.utils import plot_model

plot = plot_model(model,
           to_file='kermas_model_plot.png',
           show_shapes=True,
           show_layer_names=True)
#save model plot
print(plot)
plot.savefig('model_plot_model.png')
 